In [ ]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [ ]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/3章'
# os.chdir(path)
# ! pip install japanize-matplotlib

# ３章　可視化の仕組みを構築する１０本ノック

In [ ]:
import pandas as pd

# master data
m_store = pd.read_csv("../chapter01/input/m_store.csv")
m_area = pd.read_csv("../chapter01/input/m_area.csv")
# transacion data
order_data = pd.read_csv("../chapter01/input/tbl_order_202004.csv")

In [ ]:
order_data = pd.merge(order_data, m_store, on="store_id", how="left")
order_data = pd.merge(order_data, m_area, on="area_cd", how="left")

In [ ]:
order_data.loc[order_data["takeout_flag"] == 0, "takeout_name"] = "delivery"
order_data.loc[order_data["takeout_flag"] == 1, "takeout_name"] = "takeaway"
order_data.loc[order_data["status"] == 0, "status_name"] = "recept"
order_data.loc[order_data["status"] == 1, "status_name"] = "paid"
order_data.loc[order_data["status"] == 2, "status_name"] = "passed"
order_data.loc[order_data["status"] == 9, "status_name"] = "cancel"

In [ ]:
order_data.head()

### ノック２１：店舗を絞り込んで可視化できるようにしてみよう

> `cf.`
>
> + []()

`observe()`

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Dropdown


def order_by_store(val):
    '''
    この関数の処理はまず、
    　1. 結果エリアのクリア
       2. ドロップダウンの再描画
       3. 選択された店舗名に該当する注文データを抽出
       （ステータスコードが「paid(1)、passed(2)」を条件に付与。∵実際に売上として計上してよいデータに限定するため）
       4. 画面上に表示
    '''
    clear_output()
    display(dropdown)
    pick_data = order_data.loc[(order_data["store_name"] == val["new"]) & (order_data["status"].isin([1, 2]))]
    display(pick_data.head())

    
store_list = m_store["store_name"].tolist()
dropdown = Dropdown(options=store_list) #ドロップダウンオブジェクト生成
dropdown.observe(order_by_store, names="value") #ドロップダウンオブジェクト生成変化を検知したら関数実行
display(dropdown)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib


def graph_by_store(val):
    '''
    この関数の処理はまず、
    　1~3. order_by_storeと同様
       4. 描画用に一時的なデータフレームをコピー
       5. 日付型に再定義（object型→datetime型）
       6. 日付型のカラムをインデックス化（∵日時等でリサンプリング(再集計)を行うため）
       7. 「D」日を意味し、日単位でリサンプリングを行った結果を描画
    '''
    clear_output()
    display(dropdown2)
    pick_data = order_data.loc[(order_data["store_name"] == val["new"]) & (order_data["status"].isin([1, 2]))]
    temp = pick_data[["order_accept_date", "total_amount"]].copy()
    temp.loc[:, "order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
    temp.set_index("order_accept_date", inplace=True)
    temp.resample("D").sum().plot()

    
dropdown2 = Dropdown(options=store_list)
dropdown2.observe(graph_by_store, names="value")
display(dropdown2)

##### - reverse engineering

In [ ]:
pick_data = order_data.loc[(order_data["store_name"] == "あきる野店") & (order_data["status"].isin([1, 2]))] 
pick_data.head()

In [ ]:
temp = pick_data[["order_accept_date", "total_amount"]].copy()
temp.iloc[:, 0]= pd.to_datetime(temp["order_accept_date"])
temp.dtypes

In [ ]:
temp.set_index("order_accept_date").head()

### ノック２２：複数店舗の詳細を可視化できるようにしてみよう

In [ ]:
from ipywidgets import SelectMultiple

def order_by_multi(val):
    '''
    この関数の処理
    1. 結果エリアのクリア
    2. セレクトエリアの再描画
    3. .isin(val["new"])関数を用いて、選択された複数を取得
    4. 画面上に表示
    '''
    clear_output()
    display(select)
    pick_data = order_data.loc[(order_data["store_name"].isin(val["new"])) & (order_data["status"].isin([1, 2]))]
    display(pick_data.head())

    
# 複数の店舗を選ぶときは、ShiftキーやCt rlキーを押しながらクリック
select = SelectMultiple(options=store_list) #マルチセレクトオブジェクト生成
select.observe(order_by_multi, names="value")
display(select)

In [ ]:
def graph_by_multi(val):
    clear_output()
    display(select2)
    fig = plt.figure(figsize=(17, 4))
    plt.subplots_adjust(wspace=0.25, hspace=0.6)
    
    i = 0
    
    for trg in val["new"]:
        pick_data = order_data.loc[(order_data["store_name"] == trg)&(order_data["status"].isin([1, 2]))]
        temp = order_data[["order_accept_date", "total_amount", "store_name"]].copy()
        temp.loc[:, "order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
        temp.set_index("order_accept_date", inplace=True)
        i += 1
        ax = fig.add_subplot(1, len(val["new"]), i)
        ax.plot(temp.resample("D").sum())
        ax.set_title(trg)
        

select2 = SelectMultiple(options=store_list)
select2.observe(graph_by_multi, names="value")
display(select2)

### ノック２３：スライドバーを用いてオーダー件数を調べてみよう
> スライドパーでオーダー件数の[閾値](https://ja.wikipedia.org/wiki/%E3%81%97%E3%81%8D%E3%81%84%E5%80%A4)(しきい値:境目の値)を設定し、それより件数が少ない 多い店舗を調べてみましょう。

> `cf.`
>
> + [Pythonでリストのサイズ（要素数）を取得 - note.nkmk.me](https://note.nkmk.me/python-list-len/)

In [ ]:
from ipywidgets import IntSlider


def store_lower(val):
    clear_output()
    display(slider)
    temp = order_data.groupby("store_name")
    print(temp.size()[temp.size() < val["new"]])

slider = IntSlider(value=1100, min=1000, max=2000, step=100, description="件数：", )
slider.observe(store_lower, names="value")
display(slider)

#### - reverse engineering

In [ ]:
temp = order_data.groupby("store_name")
print(temp)

In [ ]:
temp.size()

In [ ]:
def store_upper(val):
    clear_output()
    display(slider2)
    temp = order_data.groupby("store_name")
    print(temp.size()[temp.size() > val["new"]])

slider2 = IntSlider(value=1600, min=1000, max=2000, step=100, description="件数：")
slider2.observe(store_upper, names="value")
display(slider2)

### ノック２４：トグルボタンで地域データを抽出しよう

### ノック２５：日付を指定してデータを抽出してみよう

### ノック２６：ストーリーを考えてデータを構築しよう

### ノック２７：キャンセルの理由を分析してみよう

### ノック２８：仮説を検証してみよう

### ノック２９：ストーリーをもとにパーツやデータを組み合わせてダッシュボードを作ろう

### ノック３０：ダッシュボードを改善しよう